# Data Cleaning with Pandas and Jupyter Notebooks

import needed packages

[pathlib](https://docs.python.org/3/library/pathlib.html) - Python module to handle file system paths

In [71]:
import pandas as pd
from pathlib import Path 

To make the code reusable and easier to test, and maintain a cleaner git history, I put the data cleaning functions in python .py files, and import them into the notebooks.

Use `sys.path.append` to add parent directory to system paths so the notebook can access the `scripts` directory
https://stackoverflow.com/a/64562179

In [72]:
import sys
sys.path.append(str(Path.cwd().parent))

from scripts.normalize_data import (
    normalize_columns, 
    remove_whitespace_from_column_names,
    normalize_expedition_section_cols,
    remove_bracket_text,
    remove_whitespace_from_dataframe,
    ddm2dec,
    remove_empty_unnamed_columns,
    print_df
)

## Working with multiple files

To process multiple files, we need to get the paths for all the files. 

Use Path and rglob to get all the cvs in `data_clean` directory.

In [73]:
paths = list(Path('..', 'processed_data', 'clean_data').rglob('*.csv'))
paths

[PosixPath('../processed_data/clean_data/Micropal_CSV_1/318_U1355A_Planktic_Forams.csv'),
 PosixPath('../processed_data/clean_data/Micropal_CSV_1/363-U1483A-planktic_forams.csv'),
 PosixPath('../processed_data/clean_data/Micropal_CSV_1/318_U1359A_Planktic_Forams.csv'),
 PosixPath('../processed_data/clean_data/Micropal_CSV_1/318_U1359D_Planktic_Forams.csv'),
 PosixPath('../processed_data/clean_data/Micropal_CSV_2/362_U1480E_planktic_forams.csv'),
 PosixPath('../processed_data/clean_data/Micropal_CSV_2/350_U1436B_planktic_forams.csv'),
 PosixPath('../processed_data/clean_data/Micropal_CSV_2/346_U1422A_planktic_forams.csv'),
 PosixPath('../processed_data/clean_data/Micropal_CSV_3/341_planktic_forams_U1417B.csv'),
 PosixPath('../processed_data/clean_data/Micropal_CSV_3/323_U1343B_planktic_forams.csv'),
 PosixPath('../processed_data/clean_data/Micropal_CSV_3/330_planktic_forams_U1374A.csv')]

In [74]:
len(paths)

10

## read files

I used `pandas.read_csv(path, dtype=str)` to read csv and treat all columns as strings. The reason why I used `dtype=str` is because `pandas.read_csv(path)`  will automatically convert the columns to strings, integers, floats, dates. This automatic conversion can change values in unexpected ways such as converting a column with integers and NaN into floats and NaN. 

In [76]:
path = Path('..', 'processed_data', 'clean_data', 'Micropal_CSV_2', '362_U1480E_planktic_forams.csv')

In [77]:
df = pd.read_csv(path, nrows=5 , dtype=str)
df['Pulleniatina coiling (dextral)']

0    NaN
1     20
2    NaN
3     23
4     35
Name: Pulleniatina coiling (dextral), dtype: object

In [78]:
df = pd.read_csv(path, nrows=5)
df['Pulleniatina coiling (dextral)']

0     NaN
1    20.0
2     NaN
3    23.0
4    35.0
Name: Pulleniatina coiling (dextral), dtype: float64

## viewing changes

One thing that I found helpful when data cleaning is to see the dataframe and the total number of rows and columns.

`print_df` is a custom function that calls `pd.DataFrame.shape` and `pd.DataFrame.head()`

In [80]:
path = Path('..', 'processed_data', 'clean_data', 'Micropal_CSV_2', '362_U1480E_planktic_forams.csv')
df = pd.read_csv(path, dtype=str)

print_df(df)

(23, 23)


,Sample,Top [cm],Bottom [cm],Top Depth [m],Bottom Depth [m],Datum name,Zone name (short),Zone name,Preservation,Group abundance,...,Globorotalia flexuosa,Globorotalia hessi,Globorotalia tosaensis,Globorotalia tumida,Neogloboquadrina acostaensis (dextral),Neogloboquadrina humerosa,Pulleniatina coiling (dextral),Pulleniatina coiling (sinistral),Pulleniatina finalis,Sphaeroidinella dahiscens sensu lato
0,362-U1480E-1H-CC-PAL-FORAM_CHEN,0,5,7.71,7.76,NaN,NaN,NaN,P [P43],R [A101],...,NaN,NaN,NaN,R [A102],NaN,R [A102],NaN,NaN,NaN,R [A102]
1,362-U1480E-2H-CC-PAL-FORAM_CHEN,0,5,17.54,17.59,NaN,NaN,NaN,P [P43],A [A101],...,R [A102],NaN,NaN,A [A102],NaN,R [A102],20,0,R [A102],A [A102]
2,362-U1480E-3H-1-W 130/132-FORAM_CHEN,0,2,18.6,18.62,NaN,NaN,NaN,P [P43],R [A101],...,NaN,F [A102],NaN,C [A102],NaN,F [A102],NaN,NaN,NaN,A [A102]
3,362-U1480E-3H-2-W 70/72-FORAM_CHEN,0,2,19.5,19.52,"T Globorotalia tosaensis (Gradstein et al., 2012)",PT1a [AO12],Globigerinoides fistulosus (Anthonissen and Og...,P [P43],A [A101],...,NaN,C [A102],R [A102],A [A102],NaN,NaN,23,0,R [A102],F [A102]
4,362-U1480E-3H-3-W 130/132-FORAM_CHEN,0,2,21.6,21.62,T Neogloboquadrina acostaensis (Gradstein et a...,PT1a [AO12],Globigerinoides fistulosus (Anthonissen and Og...,M [P43],A [A101],...,R [A102],NaN,R [A102],A [A102],F [A102],F [A102],35,12,F [A102],NaN


## Basic file cleanup

pandas has methods that can be used to do some basic file cleanup.

- delete dataframe column if all values are NA 

  dropna(axis='columns', how='all', inplace=True) - [pandas.DataFrame.dropna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)



- delete dataframe row if all values are NA 

  dropna(axis='index', how='all', inplace=True) - [pandas.DataFrame.dropna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)

- remove duplicate rows in dataframe 

  drop_duplicates(inplace=True) - [pandas.DataFrame.drop_duplicates](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html)

before cleanup

In [55]:
path = Path('..', 'processed_data', 'clean_data', 'Micropal_CSV_3', '341_planktic_forams_U1417B.csv')

df = pd.read_csv(path, dtype=str)
print_df(df)

(20, 41)


,Sample,Exp,Site,Hole,Core,Core-Sect,Type,Section,A/W,Top [cm],...,Neogloboquadrina pachyderma (sinistral),Neogloboquadrina pachyderma (dextral),"Neogloboquadrina pachyderma A (dextral, inflated form)","Neogloboquadrina pachyderma B (sinistral, inflated form)","Neogloboquadrina pachyderma C (sinistral, compressed form)",Orbulina universa,Comments,Ship File Links,Shore File Links,File Data
0,341-U1417B-2H-CC-PAL,341,U1417,B,2,2-CC,H,CC,PAL,0,...,D [A84],R [A84],NaN,R [A84],NaN,NaN,NaN,NaN,NaN,NaN
1,341-U1417B-4H-CC-PAL,341,U1417,B,4,4-CC,H,CC,PAL,0,...,D [A84],R [A84],NaN,NaN,NaN,NaN,sandy,NaN,NaN,NaN
2,341-U1417B-6H-CC-PAL,341,U1417,B,6,6-CC,H,CC,PAL,0,...,D [A84],NaN,NaN,NaN,NaN,NaN,diatom ooze. pyritilize foram,NaN,NaN,NaN
3,341-U1417B-9H-CC-PAL,341,U1417,B,9,9-CC,H,CC,PAL,0,...,NaN,NaN,NaN,NaN,NaN,NaN,Rich in sandy terrigenous grains with presence...,NaN,NaN,NaN
4,341-U1417B-10H-CC-PAL,341,U1417,B,10,10-CC,H,CC,PAL,0,...,D [A84],R [A84],NaN,NaN,NaN,NaN,"Rich in planktic foraminifers, pebbles, pyrite",NaN,NaN,NaN


after cleanup

In [56]:
df.dropna(axis='columns', how='all', inplace=True)  
df.dropna(axis='index', how='all', inplace=True)
df.drop_duplicates(inplace=True)

print_df(df)

(17, 26)


,Sample,Exp,Site,Hole,Core,Core-Sect,Type,Section,A/W,Top [cm],...,Preservation,Group abundance,Globigerina umbilicata,Globigerinita glutinata,Neogloboquadrina inglei,Neogloboquadrina kagaensis,Neogloboquadrina pachyderma (sinistral),Neogloboquadrina pachyderma (dextral),"Neogloboquadrina pachyderma B (sinistral, inflated form)",Comments
0,341-U1417B-2H-CC-PAL,341,U1417,B,2,2-CC,H,CC,PAL,0,...,G [P15],R [A83],D [A84],R [A84],NaN,NaN,D [A84],R [A84],R [A84],NaN
1,341-U1417B-4H-CC-PAL,341,U1417,B,4,4-CC,H,CC,PAL,0,...,G [P15],C [A61],A [A84],P [A84],NaN,NaN,D [A84],R [A84],NaN,sandy
2,341-U1417B-6H-CC-PAL,341,U1417,B,6,6-CC,H,CC,PAL,0,...,P [P15],P [A83],D [A84],NaN,NaN,NaN,D [A84],NaN,NaN,diatom ooze. pyritilize foram
3,341-U1417B-9H-CC-PAL,341,U1417,B,9,9-CC,H,CC,PAL,0,...,NaN,B [A83],NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rich in sandy terrigenous grains with presence...
4,341-U1417B-10H-CC-PAL,341,U1417,B,10,10-CC,H,CC,PAL,0,...,G [P15],A [A83],D [A84],P [A84],P [A84],NaN,D [A84],R [A84],NaN,"Rich in planktic foraminifers, pebbles, pyrite"


Use for loop and paths to run basic cleanup on all files

In [57]:
for path in paths:
    df = pd.read_csv(path, dtype=str)
    
    df.dropna(axis='columns', how='all', inplace=True)  
    df.dropna(axis='index', how='all', inplace=True)
    df.drop_duplicates(inplace=True)
    
    df.to_csv(path, index=False)

## Git

You should `git commit` after each cleanup step so that you can keep track of the changes and undo changes in needed.

```
$ git add .
$ git commit -m 'do basic file cleanup on all files'
```

## remove leading and trailing white spaces

I created a custom function `remove_whitespace_from_dataframe` to remove leading and trailing white spaces for the headers and all the rows. 

Since I wanted to remove white spaces from the headers, I used `read_csv(header=None)` and `to_csv(header=False)` so that pandas treat the first row like any other row.

In [58]:
for path in paths:
    df = pd.read_csv(path, dtype=str, header=None)
    
    remove_whitespace_from_dataframe(df)
    
    df.to_csv(path, index=False, header=False)

## Normalizing columns names

For the expedition 312 and later, the researchers for each expedition  determined the format of their data files. This resulted in a lot of variability in the file columns.  Another challenge with parsing the files is that each taxa is stored as separate column in the files.

### get all unique column names

Since I only want the header names, use `nrow=0` with `read_csv`. 

In [68]:
pd.read_csv(paths[1], dtype=str, nrows=0)


,Sample,Top [cm],Bottom [cm],Top Depth [m],Bottom Depth [m],Zone name (short),Zone name,Additional zone name (short),Additional zone name,Preservation,...,Globoturborotalita decoraperta _T and B,Globoturborotalita rubescens,Sphaeroidinellopsis seminulina _T_ _PL4,Sphaeroidinellopsis kochi _T,Globigerinoidesella fistulosa _T and B_ _Pt1a,Globigerinoides ruber _pink_ T,Globigerinoides extremus _T and B,Globigerinoides obliquus _T,Globigerinella calida _B,Sample comment


I used `pandas.DataFrame.columns()` and python `set` to get all the unique columns fo all the files.

In [81]:
all_columns = set()
for path in paths:
    df = pd.read_csv(path, dtype=str, nrows=0)
    
    all_columns.update(df.columns)
    
len(all_columns)

158

In [82]:
all_columns

{'A/W',
 'Additional zone name',
 'Additional zone name (short)',
 'Bottom Depth [m]',
 'Bottom Depth[m] [m]',
 'Bottom [cm]',
 'Bottom[cm] [cm]',
 'Candeina nitida',
 'Comment (general)',
 'Comments',
 'Core',
 'Core-Sect',
 'Datum name',
 'Dentoglobigerina altispira _T_ _PL5',
 'Dentoglobigerina altispira _T_ _PL5_',
 'Dextral:Sinistral _P. obliquiloculata_',
 'Dextral:Sinistral _P. praecursor_',
 'Dextral:Sinistral _P. primalis_',
 'Exp',
 'Extra Sample ID Data',
 'Fragmentation',
 'Fragmentation rank [auto-pop]',
 'Genus/species (upper zone)',
 'Genus/species lower zone)',
 'Globigerina bulloides',
 'Globigerina cf. woodi',
 'Globigerina falconensis',
 'Globigerina umbilicata',
 'Globigerinella aequilateralis',
 'Globigerinella calida',
 'Globigerinella calida _B',
 'Globigerinella calida _B_',
 'Globigerinella obesa',
 'Globigerinita glutinata',
 'Globigerinita parkerae',
 'Globigerinita uvula',
 'Globigerinoides bulloideus',
 'Globigerinoides conglobatus',
 'Globigerinoides extre

I then manually separate taxa names from other headers so that we could do some more processing on the taxa.

In [62]:
taxa_columns = {
 'Candeina nitida',
 'Dentoglobigerina altispira _T_ _PL5',
 'Dentoglobigerina altispira _T_ _PL5_',
 'Dextral:Sinistral _P. obliquiloculata_',
 'Dextral:Sinistral _P. praecursor_',
 'Dextral:Sinistral _P. primalis_',
 'Globigerina bulloides',
 'Globigerina cf. woodi',
 'Globigerina falconensis',
 'Globigerina umbilicata',
 'Globigerinella aequilateralis',
 'Globigerinella calida',
 'Globigerinella calida _B',
 'Globigerinella calida _B_',
 'Globigerinella obesa',
 'Globigerinita glutinata',
 'Globigerinita parkerae',
 'Globigerinita uvula',
 'Globigerinoides bulloideus',
 'Globigerinoides conglobatus',
 'Globigerinoides extremus _T and B',
 'Globigerinoides extremus _T and B_',
 'Globigerinoides fistulosus',
 'Globigerinoides obliquus _T',
 'Globigerinoides obliquus _T_',
 'Globigerinoides quadrilobatus',
 'Globigerinoides ruber',
 'Globigerinoides ruber (pink)',
 'Globigerinoides ruber (white)',
 'Globigerinoides ruber _pink_ T',
 'Globigerinoides ruber _pink_ _T_',
 'Globigerinoides sacculifer',
 'Globigerinoides sacculifer (without sack)',
 'Globigerinoides tenellus',
 'Globigerinoides trilobus',
 'Globigerinoidesella fistulosa _T and B_ _Pt1a',
 'Globigerinoidesella fistulosa _T and B_ _Pt1a_',
 'Globoconella miozea',
 'Globorotalia (Globoconella) inflata',
 'Globorotalia (Globorotalia) tumida tumida',
 'Globorotalia (Hirsutella) hirsuta',
 'Globorotalia (Hirsutella) scitula',
 'Globorotalia (Truncorotalia) crossaformis',
 'Globorotalia (Truncorotalia) truncatulinoides',
 'Globorotalia anfracta',
 'Globorotalia crassaformis',
 'Globorotalia crassaformis sensu lato',
 'Globorotalia flexuosa',
 'Globorotalia flexuosa _T and B_',
 'Globorotalia hessi',
 'Globorotalia hessi _B_',
 'Globorotalia hirsuta',
 'Globorotalia inflata',
 'Globorotalia limbata _B',
 'Globorotalia limbata _B_',
 'Globorotalia limbata _T_',
 'Globorotalia margaritae _T and B_ _PL3',
 'Globorotalia margaritae _T and B_ _PL3_',
 'Globorotalia menardii',
 'Globorotalia multicamerata _T',
 'Globorotalia multicamerata _T_',
 'Globorotalia plesiotumida _B_ _M13b_',
 'Globorotalia plesiotumida _T',
 'Globorotalia plesiotumida _T_',
 'Globorotalia pseudomiocenica _T_ _PL6',
 'Globorotalia pseudomiocenica _T_ _PL6_',
 'Globorotalia scitula',
 'Globorotalia tosaensis',
 'Globorotalia tosaensis _T and B_ _Pt1b',
 'Globorotalia tosaensis _T and B_ _Pt1b_',
 'Globorotalia truncatulinoides',
 'Globorotalia truncatulinoides _B',
 'Globorotalia truncatulinoides _B_',
 'Globorotalia tumida',
 'Globorotalia tumida _B_ _PL1a_',
 'Globoturborotalita apertura _T and B',
 'Globoturborotalita apertura _T and B_',
 'Globoturborotalita decoraperta _T and B',
 'Globoturborotalita decoraperta _T and B_',
 'Globoturborotalita rubescens',
 'Neogloboquadrina acostaensis',
 'Neogloboquadrina acostaensis (dextral)',
 'Neogloboquadrina cf. pachyderma',
 'Neogloboquadrina dutertrei',
 'Neogloboquadrina humerosa',
 'Neogloboquadrina incompta (dextral)',
 'Neogloboquadrina inglei',
 'Neogloboquadrina kagaensis',
 'Neogloboquadrina nympha',
 'Neogloboquadrina pachyderma (dextral)',
 'Neogloboquadrina pachyderma (sin)',
 'Neogloboquadrina pachyderma (sinistral)',
 'Neogloboquadrina pachyderma B (sinistral, inflated form)',
 'Neogloboquadrina pachyderma(dex)',
 'Orbulina universa',
 'Pulleniatina coiling (dextral)',
 'Pulleniatina coiling (sinistral)',
 'Pulleniatina finalis',
 'Pulleniatina finalis _B',
 'Pulleniatina finalis _B_',
 'Pulleniatina obliquiloculata',
 'Pulleniatina obliquiloculata (D)',
 'Pulleniatina praecursor',
 'Pulleniatina praespectabilis',
 'Pulleniatina primalis  _Tand B',
 'Pulleniatina primalis  _Tand B_',
 'Sphaeroidinella dahiscens sensu lato',
 'Sphaeroidinella dehiscens',
 'Sphaeroidinella dehiscens s.l.',
 'Sphaeroidinella dehiscens sensu lato _B_',
 'Sphaeroidinellopsis kochi _T',
 'Sphaeroidinellopsis kochi _T_',
 'Sphaeroidinellopsis seminulina _T_ _PL4',
 'Sphaeroidinellopsis seminulina _T_ _PL4_'
}

In [63]:
len(taxa_columns)

114

Since both `all_columns` and `taxa_columns` are sets, we can subtract them to get the difference.

In [64]:
nontaxa_columns = all_columns - taxa_columns

nontaxa_columns

{'A/W',
 'Additional zone name',
 'Additional zone name (short)',
 'Bottom Depth [m]',
 'Bottom Depth[m] [m]',
 'Bottom [cm]',
 'Bottom[cm] [cm]',
 'Comment (general)',
 'Comments',
 'Core',
 'Core-Sect',
 'Datum name',
 'Exp',
 'Extra Sample ID Data',
 'Fragmentation',
 'Fragmentation rank [auto-pop]',
 'Genus/species (upper zone)',
 'Genus/species lower zone)',
 'Group Abundance',
 'Group abundance',
 'Group abundance (%)',
 'Hole',
 'Label ID',
 'PF Preservation',
 'Preservation',
 'Sample',
 'Sample comment',
 'Section',
 'Site',
 'Top Depth [m]',
 'Top Depth[m] [m]',
 'Top [cm]',
 'Top[cm] [cm]',
 'Type',
 'Type (lower zone)',
 'Upper boundary age av. [Ma]',
 'Zone author (year)',
 'Zone group',
 'Zone name',
 'Zone name (short)',
 'pc_abundance_name_mode',
 'pc_fossil_group',
 'pc_fossil_name',
 'physical_constituent_name'}

In [14]:
len(nontaxa_columns)

50

### create taxa and non-taxa file

I saved the the taxa and nontaxa headers to csv so that I can access them later.

In [15]:
taxa_df = pd.DataFrame(taxa_columns, columns=['verbatim_name'])
taxa_df.sort_values('verbatim_name', inplace=True)

print_df(taxa_df)

(141, 1)


,verbatim_name
77,Beella digitata
29,Candeina nitida
51,Dentoglobigerina altispira
76,Dentoglobigerina altispira _T_ _PL5
96,Dentoglobigerina altispira _T_ _PL5_


In [16]:
path = Path('..', 'processed_data', 'drafts', 'taxa_list.csv')
taxa_df.to_csv(path, index=False)

In [17]:
non_taxa_df = pd.DataFrame(nontaxa_columns, columns=['field'])
non_taxa_df.sort_values('field', inplace=True)

print_df(non_taxa_df)

(50, 1)


,field
46,A/W
25,Additional zone name
49,Additional zone name (short)
45,Bottom Depth [m]
16,Bottom Depth[m] [m]


In [18]:
path = Path('..', 'processed_data', 'drafts', 'nontaxa_list.csv')
non_taxa_df.to_csv(path, index=False)

### combine headers

In [19]:
for path in paths:
    df = pd.read_csv(path, dtype=str)    
    cols = df.columns
    
    variants = ['Bottom Depth [m]', 'Bottom depth [m]', 'Bottom Depth[m] [m]']
    replacement = 'Bottom Depth [m]'
    cols = normalize_columns(variants, replacement, cols)

    variants = ['Bottom [cm]', 'Bottom[cm] [cm]']
    replacement = 'Bottom [cm]'
    cols = normalize_columns(variants, replacement, cols)

    
    variants = ['Label ID', 'Sample']
    replacement = 'Sample'
    cols = normalize_columns(variants, replacement, cols)

    variants = ['Top Depth [m]', 'Top depth [m]', 'Top Depth[m] [m]']
    replacement = 'Top Depth [m]'
    cols = normalize_columns(variants, replacement, cols)

    variants = ['Top [cm]', 'Top[cm] [cm]']
    replacement = 'Top [cm]'
    cols = normalize_columns(variants, replacement, cols)
    
    variants = ['Comment', 'Comment (general)', 'Comments', 'Sample comment']
    replacement = 'Comment'
    cols = normalize_columns(variants, replacement, cols)
    
    variants = ['Genus/species lower zone)']
    replacement = 'Genus/species (lower zone)'
    cols = normalize_columns(variants, replacement, cols)
    
    variants = ['Group Abundance', 'Group abundance', 'Group abundance (%)']
    replacement = 'Group Abundance'
    cols = normalize_columns(variants, replacement, cols)
    
    variants = ['PF Preservation', 'PF preservation']
    replacement = 'PF preservation'
    cols = normalize_columns(variants, replacement, cols)
    
    df.columns = cols 
    df.to_csv(path, index=False)


## Clean up row values

In [ ]:
For this data set, we wanted the remove the 

In [20]:
for path in paths:
    df = pd.read_csv(path, dtype=str)
    
    df = remove_bracket_text(df)
    
    df.to_csv(path, index=False)

## convert sample column to  Exp...A/W colums

In [21]:
for path in paths:
    df = pd.read_csv(path, dtype=str)   
    
    df = normalize_expedition_section_cols(df)
    
    df.to_csv(path, index=False) 

## check mandatory columns

In [22]:
required_columns = {
 'A/W',
 'Bottom [cm]',
 'Bottom Depth [m]',
 'Core',
 'Exp',
 'Hole',
 'Sample',
 'Section',
 'Site',
 'Top [cm]',
 'Top Depth [m]',
 'Type'
}

In [23]:
for path in paths:
    df = pd.read_csv(path, dtype=str)    
    cols = set(df.columns)
    diff = required_columns - cols
    
    if(len(diff) > 0):
        print(path)
        print(required_columns - cols)
    